In [1]:
import tensorflow as tf
import tensorflow_decision_forests as tfdf
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import pandas as pd

import shap

In [2]:
wine_qt_path = '../data/WineQT.csv'
RANDOM_SEED = 492

In [3]:
wine_df = pd.read_csv(wine_qt_path)

In [4]:
print(wine_df.head())

   fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0            7.4              0.70         0.00             1.9      0.076   
1            7.8              0.88         0.00             2.6      0.098   
2            7.8              0.76         0.04             2.3      0.092   
3           11.2              0.28         0.56             1.9      0.075   
4            7.4              0.70         0.00             1.9      0.076   

   free sulfur dioxide  total sulfur dioxide  density    pH  sulphates  \
0                 11.0                  34.0   0.9978  3.51       0.56   
1                 25.0                  67.0   0.9968  3.20       0.68   
2                 15.0                  54.0   0.9970  3.26       0.65   
3                 17.0                  60.0   0.9980  3.16       0.58   
4                 11.0                  34.0   0.9978  3.51       0.56   

   alcohol  quality  Id  
0      9.4        5   0  
1      9.8        5   1  
2      9

In [5]:
y_series = wine_df['Id']
y = pd.DataFrame(y_series, columns=['Id'])
features = [col for col in wine_df.columns if col != 'Id']
X = wine_df[features]

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [7]:
X_train = X_train.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)

In [8]:
subset_indices = np.random.choice(X_train.index, size=len(X_train), replace=False)

X_train_subset = X_train.loc[subset_indices]
y_train_subset = y_train.loc[subset_indices]

print(X_train_scaled.head())

[ 66 425 602 345 123 464 387 125 138 670 828  52  32 268 854 310 771   4
 407 525 755 312 720 389 458 252 696   0 422 770 408 675 248 864 241 214
 472 667 716 172  78 821 113 541 149 582 240 704 794 493   5 155 901 166
 522 153 740 282  48 347 270 851 173 665 912 612 610 867 700 316 372 116
 567 218 461 401 895 690 295 226 221 580 266 447  45 517 598 882 431  65
 911 110 342 643 719 399 641  38 833 193   9 100 676 708 790 793 743 661
  64 565 376 737 210 605 774 573 322 482 766 233 136 614  29 480 677 238
  23 371 177 109 850 157 380  28 339 260 596  99 608 292 375 899 879  75
 710 205 179 760 653 118 906 465 470   3 553 560 786 148 232  73 457 782
 681 117  20 620 492 562 432  81 627 888 644 369 132 607 540  76  19 409
 544 511 519 817 831 566 307 291 648 393 351 139 526 732 414 591 243 377
  35 869 358 309 191 739 858 231 381 750 832 634 516 555 741 880 394 305
 228 468 344 335 723 239 780 826 859 183 767  44 730 346 829 787 114 159
 278  25 843 890 485 161 474 104 146 518 763 534 14

In [9]:
X_train_subset = X_train_subset.reset_index(drop=True)
y_train_subset = y_train_subset.reset_index(drop=True)

In [10]:
X_train_tensor = tf.constant(X_train_subset.values, dtype=tf.float32)
y_train_tensor = tf.constant(y_train_subset.values, dtype=tf.float32)
print(y_train_tensor.shape)

(914, 1)


In [15]:
class GradientBoostedTreesEnsembleClassifier(tf.keras.Model):
    def __init__(self, n_trees=100, max_depth=3, n_estimators=100, subportion = 0.8, sample_method='RANDOM'): 
        super(GradientBoostedTreesEnsembleClassifier, self).__init__()
        self.n_trees = n_trees
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.subportion = subportion
        self.sample_method = sample_method
        self.estimators = []

    def build_estimator(self):
        model = tfdf.keras.GradientBoostedTreesModel(
            task=tfdf.keras.Task.CLASSIFICATION,
            num_trees=np.random.randint(self.n_trees/5, self.n_trees+1), # self.n_trees, #
            max_depth=self.max_depth, #np.random.randint(self.max_depth/2, self.max_depth+1), # self.max_depth, # 
            validation_ratio=0.1,
            subsample=self.subportion,
            sampling_method=self.sample_method,
            random_seed=self.seed # try to set this first. Maybe this is enough
        )
        return model
    
    # if not, the number of trees/depth different for each estimator

    def fit(self, X, y):
        
        for seed in np.random.randint(0, 1000000, size=self.n_estimators):
            self.seed = int(seed)
            subset_size = int(self.subportion * len(X))
            
            # Create a subset of the dataset using sampled indices
            subset_indices = np.random.choice(X.index, size=subset_size, replace=False)
            X_train = X.loc[subset_indices].reset_index(drop=True)
            y_train = y.loc[subset_indices].reset_index(drop=True)
            
            X_tensor = tf.constant(X_train.values, dtype=tf.float32)
            y_tensor = tf.constant(y_train.values, dtype=tf.float32)
        
            
            # subset_dataset = (X_tensor, y_tensor)
            
            # building each estimator
            estimator = self.build_estimator()
            estimator.fit(X_tensor,y_tensor)
            self.estimators.append(estimator)
    
    #something's not right here
    def predict(self, X): # use all estimators
        predictions = []
        # sampled_estimators = np.random.choice(self.estimators, size=n_samples, replace=False)
        for estimator in self.estimators:
            estimator_predictions = []
            estimator_predictions = estimator.predict(X)
            predictions.append(estimator.predict(X))
        predictions_stacked = tf.stack(predictions, axis=-1)
        
        mean_prediction = tf.reduce_mean(predictions_stacked, axis=-1)
        variance_prediction = tf.math.reduce_std(predictions_stacked, axis=-1)
        return mean_prediction, variance_prediction


In [16]:
gbt_model = GradientBoostedTreesEnsembleClassifier(
    n_trees=50, 
    max_depth=5, 
    n_estimators=80, 
    subportion=0.8, 
    sample_method='RANDOM'
)

In [17]:
gbt_model.fit(X_train, y_train)

Use /var/folders/3r/pzt2_p_x2xdfl2057nxkp3nm0000gn/T/tmpbo8n_wgb as temporary training directory
Reading training dataset...
Training dataset read in 0:00:00.105307. Found 731 examples.
Training model...


[WARNING 24-05-16 11:12:13.9649 CEST gradient_boosted_trees.cc:1840] "goss_alpha" set but "sampling_method" not equal to "GOSS".
[WARNING 24-05-16 11:12:13.9649 CEST gradient_boosted_trees.cc:1851] "goss_beta" set but "sampling_method" not equal to "GOSS".
[WARNING 24-05-16 11:12:13.9649 CEST gradient_boosted_trees.cc:1865] "selective_gradient_boosting_ratio" set but "sampling_method" not equal to "SELGB".
[INFO 24-05-16 11:12:33.7830 CEST kernel.cc:1233] Loading model from path /var/folders/3r/pzt2_p_x2xdfl2057nxkp3nm0000gn/T/tmpbo8n_wgb/model/ with prefix 06039db898be411f


Model trained in 0:00:20.343895
Compiling model...
Model compiled.


[INFO 24-05-16 11:12:34.3869 CEST decision_forest.cc:734] Model loaded with 17556 root(s), 343132 node(s), and 12 input feature(s).
[INFO 24-05-16 11:12:34.3870 CEST abstract_model.cc:1344] Engine "GradientBoostedTreesGeneric" built
[INFO 24-05-16 11:12:34.3870 CEST kernel.cc:1061] Use fast generic engine


Use /var/folders/3r/pzt2_p_x2xdfl2057nxkp3nm0000gn/T/tmptcueff7p as temporary training directory
Reading training dataset...
Training dataset read in 0:00:00.094672. Found 731 examples.
Training model...


...

[INFO 24-05-16 11:44:42.9973 CEST decision_forest.cc:734] Model loaded with 17578 root(s), 327992 node(s), and 12 input feature(s).
[INFO 24-05-16 11:44:42.9973 CEST abstract_model.cc:1344] Engine "GradientBoostedTreesGeneric" built
[INFO 24-05-16 11:44:42.9973 CEST kernel.cc:1061] Use fast generic engine


Use /var/folders/3r/pzt2_p_x2xdfl2057nxkp3nm0000gn/T/tmpt76_ydqi as temporary training directory
Reading training dataset...
Training dataset read in 0:00:00.094824. Found 731 examples.
Training model...


[WARNING 24-05-16 11:44:43.2400 CEST gradient_boosted_trees.cc:1840] "goss_alpha" set but "sampling_method" not equal to "GOSS".
[WARNING 24-05-16 11:44:43.2400 CEST gradient_boosted_trees.cc:1851] "goss_beta" set but "sampling_method" not equal to "GOSS".
[WARNING 24-05-16 11:44:43.2400 CEST gradient_boosted_trees.cc:1865] "selective_gradient_boosting_ratio" set but "sampling_method" not equal to "SELGB".
[INFO 24-05-16 11:45:02.0084 CEST kernel.cc:1233] Loading model from path /var/folders/3r/pzt2_p_x2xdfl2057nxkp3nm0000gn/T/tmpt76_ydqi/model/ with prefix bd0fc29bb1e34bc5


Model trained in 0:00:19.275221
Compiling model...
Model compiled.


[INFO 24-05-16 11:45:02.5817 CEST decision_forest.cc:734] Model loaded with 17556 root(s), 337582 node(s), and 12 input feature(s).
[INFO 24-05-16 11:45:02.5818 CEST abstract_model.cc:1344] Engine "GradientBoostedTreesGeneric" built
[INFO 24-05-16 11:45:02.5818 CEST kernel.cc:1061] Use fast generic engine


In [18]:
mean_predict, variance_predict = gbt_model.predict(X_test)

8/8 [==============================] - 0s 8ms/step


2024-05-16 12:34:53.784720: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: INVALID_ARGUMENT: Shapes of all inputs must match: values[0].shape = [229,1596] != values[1].shape = [229,1588]


InvalidArgumentError: {{function_node __wrapped__Pack_N_80_device_/job:localhost/replica:0/task:0/device:CPU:0}} Shapes of all inputs must match: values[0].shape = [229,1596] != values[1].shape = [229,1588] [Op:Pack] name: stack

In [ ]:
y_test_eval = np.array(y_test).flatten()
y_pred_eval = mean_predict.numpy().flatten()

In [ ]:
PLOT = 30

In [ ]:
y_plot = y_test[:PLOT]
x_axis = np.arange(len(y_plot))
y_mean_predict = mean_predict[:PLOT].numpy()
y_var_predict = variance_predict[:PLOT].numpy()

In [ ]:
plt.figure(figsize=(8, 6))
plt.xlim(-1,PLOT)
plt.scatter(x_axis, y_plot, color='lightblue', label="Ground Truth")
plt.scatter(x_axis, y_mean_predict.flatten(), color='pink', alpha=0.8, label='Mean Prediction')
# plt.errorbar(x_axis, y_mean_predict.flatten(), yerr=root_mse, fmt='none', ecolor='pink', label="Root MSE")
plt.errorbar(x_axis, y_mean_predict.flatten(), yerr=y_var_predict.flatten(), fmt='none', color='darkred', alpha=0.8, label="Uncertainty")
plt.ylabel('prices')
plt.legend()
plt.grid(True)  # Enable grid for better visualization
plt.show()

In [ ]:
# predict gives the class number
# make a histogram of the class prediction
# entropy to measure the uncertainty